# 5. Forecasting

[index](../Index.ipynb) | [prev](./04.DataCollection.ipynb) | [next](./06.AnomalyDetection.ipynb)

The idea to generate a forecast for the detected objects did not exist in this project from the beginning, when the idea was to build a toy-project to use a Raspberry Pi for real time object detection.

The step above was accomplished within six months and an idea occurred to me during a student meetup in the university: Can detected objects be used as a dataset to predict object counts?

This tought has changed my perspective about the project, and since then I began to look at images as a valuable data source.

Forecasting expected object counts can be very beneficial not only for our own curiosity, but also for anomaly detection. For example, if the predicted number of people at 3PM on a weekend-day is 5 and we have already observed 15 people, then something unusual must be happening and perhaps we would like to be notified about it.

This Notebook dives into the detail of the steps taken to generate such a forecast:
- data extraction and clean up of raw images
- approach to counting objects
- exploratory data analysis
- forecasting